In [ ]:
#Only run this in colab
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Github/Dimension
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

In [1]:
import rubics
import numpy as np
from student import student
from classroom import classroom
from teacher import teacher
from rubics_neural_networks import student_network

task = rubics.rubics_task
setup = rubics.rubics_setup

student_network = student_network()
student_template = student(task, 40,student_network)
t = teacher(setup)
c = classroom(task, setup, t, student_template, n_students=1, max_steps=10)
c.run_training_batch(100)


Finished step 0, 48 out of 50 remain open.
Finished step 1, 43 out of 50 remain open.
Finished step 2, 43 out of 50 remain open.
Finished step 3, 43 out of 50 remain open.
Finished step 4, 43 out of 50 remain open.
Finished step 5, 43 out of 50 remain open.
Finished step 6, 43 out of 50 remain open.
Finished step 7, 43 out of 50 remain open.
Finished step 8, 43 out of 50 remain open.
Finished step 9, 43 out of 50 remain open.
Epoch 1/3
14/14 [==============================] - 2s 6ms/step - loss: 108.1565 - eval_output_loss: 6.7721e-04 - policy_output_loss: 2.2841
Epoch 2/3
14/14 [==============================] - 0s 7ms/step - loss: 81.9383 - eval_output_loss: 3.2554e-05 - policy_output_loss: 0.8273
Epoch 3/3
14/14 [==============================] - 0s 6ms/step - loss: 47.9552 - eval_output_loss: 1.4527e-05 - policy_output_loss: 0.4377


In [3]:
for i in range(3):
    c.run_training_batch(50)

Finished step 0, 47 out of 50 remain open.
Finished step 1, 36 out of 50 remain open.
Finished step 2, 34 out of 50 remain open.
Finished step 3, 33 out of 50 remain open.
Finished step 4, 33 out of 50 remain open.
Finished step 5, 33 out of 50 remain open.
Finished step 6, 33 out of 50 remain open.
Finished step 7, 33 out of 50 remain open.
Finished step 8, 33 out of 50 remain open.
Finished step 9, 33 out of 50 remain open.
Finished step 10, 33 out of 50 remain open.
Finished step 11, 33 out of 50 remain open.
Finished step 12, 33 out of 50 remain open.
Finished step 13, 33 out of 50 remain open.
Finished step 14, 33 out of 50 remain open.
Epoch 1/3
17/17 [==============================] - 0s 7ms/step - loss: 5.0745 - eval_output_loss: 0.0017 - policy_output_loss: 0.1920
Epoch 2/3
17/17 [==============================] - 0s 7ms/step - loss: 4.4937 - eval_output_loss: 1.8616e-04 - policy_output_loss: 0.1612
Epoch 3/3
17/17 [==============================] - 0s 7ms/step - loss: 4.3541 